In [5]:
import xml.etree.ElementTree as ET
import pandas as pd
with open('./arXiv_src_manifest.xml', 'r') as f:
    mani = ET.parse(f)

In [15]:
root = mani.getroot()
def parse_element(elem):
    return_dict = {}
    for e in elem:
        return_dict[e.tag] = e.text
    return return_dict
def parse_root(root):
        return [parse_element(child) for child in iter(root) if child.tag != 'timestamp']

filedf = pd.DataFrame(parse_root(root))
filedf[['num_items', 'size']] = filedf[['num_items', 'size']].astype(int)
filedf['filename'] = filedf['filename'].astype(str)

In [16]:
import hashlib 

In [71]:
hash_md5 = hashlib.md5()
with open('../arXiv_src_1804_001.tar', "rb") as f:
    for chunk in iter(lambda: f.read(4096), b""):
        hash_md5.update(chunk)
hash_md5.hexdigest()

'fe34df50d3426c745ea9a8b5ed924859'

In [72]:
filedf.loc[filedf['filename'].str.contains('1804_001')]

,content_md5sum,filename,first_item,last_item,md5sum,num_items,seq_num,size,timestamp,yymm
1742,d2ca6595ddb8055262ea1c6adf4cd4e3,src/arXiv_src_1804_001.tar,1804.00001,1804.00262,fe34df50d3426c745ea9a8b5ed924859,262,1,526207253,2018-06-05 04:56:42,1804


In [7]:
S = filedf['size']

In [9]:
S.sum()

924382365733

In [46]:
import subprocess
P = subprocess.run(['/bin/s3cmd', 'get', '--requester-pays', 's3://arxiv/src/arXiv_src_1804_006.tar', '..'], 
                   stderr=subprocess.PIPE,
                   stdout=subprocess.PIPE)

In [41]:
P.stdout

b"download: 's3://arxiv/src/arXiv_src_1804_006sab.tar' -> '../arXiv_src_1804_006sab.tar'  [1 of 1]\ndownload: 's3://arxiv/src/arXiv_src_1804_006sab.tar' -> '../arXiv_src_1804_006sab.tar'  [1 of 1]\n"

In [42]:
P.stderr

b'ERROR: S3 error: 404 (NoSuchKey): The specified key does not exist.\n'

In [45]:
P.check_returncode()

CalledProcessError: Command '['/bin/s3cmd', 'get', '--requester-pays', 's3://arxiv/src/arXiv_src_1804_006sab.tar', '..']' returned non-zero exit status 12.

In [19]:
df = filedf.head()
notdf = pd.DataFrame([{'filename': 'src/arXiv_src_0001_001.tar'}])

In [74]:
idx = df.filename.isin(notdf.filename)
df[~idx].iloc[0]

content_md5sum    d90df481661ccdd7e8be883796539743
filename                src/arXiv_src_0002_001.tar
first_item                         astro-ph0002001
last_item                          quant-ph0002094
md5sum            4592ab506cf775afecf4ad560d982a00
num_items                                     2365
seq_num                                          1
size                                     227036528
timestamp                      2010-12-23 00:18:09
yymm                                          0002
Name: 1, dtype: object

In [55]:
df.filename.isin(notdf.filename)

0    False
1     True
2    False
3    False
4    False
Name: filename, dtype: bool

In [57]:
df.to_csv('head_df.csv')

In [58]:
notdf.to_csv('notdf.csv')

In [35]:
nondf = pd.read_csv('/home/luis/mountpoint/arXiv_src/downloaded_log.csv',index_col=0)
nondf.append(pd.DataFrame([{'filename':'hola'}]), ignore_index=True)

/usr/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


,Unnamed: 0.1,filename
0,NaN,hola


In [63]:
import os
os.path.ismount('/home/luis/mnt')

False

In [39]:
notdf

,filename
0,src/arXiv_src_0001_001.tar


In [40]:
#filedf.to_csv('/home/luis/mountpoint/allfiles.csv')
notdf.to_csv('/home/luis/mountpoint/arXiv_src/downloaded_log.csv')

In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
from dload import DownloadMan, parse_manifest
mountpoint = '/mnt/arXiv_src/'
allfiles = 'allfiles.csv'
doun = 'downloaded_log.csv'
error_log = 'error.log'
D = DownloadMan(mountpoint, allfiles, doun, error_log)

In [5]:
D.get_src_manifest()

Download of file src/arXiv_src_manifest.xml successful


0

In [10]:
D.check_md5()

False

In [7]:
new_all = parse_manifest('/mnt/arXiv_src/src/arXiv_src_manifest.xml')

In [12]:
fname = 'src/arXiv_src_1607_002.tar'
D.md5sum(fname)

'1b621f41cd46b9d3388351ba9e3aefaf'

In [13]:
D.allfiles_df[D.allfiles_df.filename == fname]

,content_md5sum,filename,first_item,last_item,md5sum,num_items,seq_num,size,timestamp,yymm
1218,8979b2424c6d31c8be791aa675ef05ba,src/arXiv_src_1607_002.tar,1607.00463,1607.00988,a29303e80c4f753e17209a9f6238f2c5,526,2,545216578,2017-07-05 05:41:14,1607


In [14]:
new_all[new_all.filename == fname]

,content_md5sum,filename,first_item,last_item,md5sum,num_items,seq_num,size,timestamp,yymm
1218,2446538d56ddee4f0e28ee65937b6189,src/arXiv_src_1607_002.tar,1607.00463,1607.00988,1b621f41cd46b9d3388351ba9e3aefaf,526,2,556653925,2018-07-05 06:37:36,1607


In [22]:
D.allfiles_df[D.allfiles_df.filename == fname].md5sum.iloc[0]

'5c0c499b3cc7c4961e522aa55e8974a5'

In [24]:
sum([D.filesize(fn[0]) for fn in list(D.downloaded_df.values)])/1e9

116.614211313

In [ ]:
keep_going = 0
while keep_going == 0:
    keep_going = D.get_next()
    if D.check_md5():
        pass
    else:
        break

Dowloand of file src/arXiv_src_0003_001.tar successful
Dowloand of file src/arXiv_src_0004_001.tar successful
Dowloand of file src/arXiv_src_0005_001.tar successful
Dowloand of file src/arXiv_src_0006_001.tar successful
Dowloand of file src/arXiv_src_0007_001.tar successful
Dowloand of file src/arXiv_src_0008_001.tar successful
Dowloand of file src/arXiv_src_0009_001.tar successful
Dowloand of file src/arXiv_src_0010_001.tar successful
Dowloand of file src/arXiv_src_0011_001.tar successful
Dowloand of file src/arXiv_src_0012_001.tar successful
Dowloand of file src/arXiv_src_0101_001.tar successful
Dowloand of file src/arXiv_src_0102_001.tar successful
Dowloand of file src/arXiv_src_0103_001.tar successful
Dowloand of file src/arXiv_src_0104_001.tar successful
Dowloand of file src/arXiv_src_0105_001.tar successful
Dowloand of file src/arXiv_src_0106_001.tar successful
Dowloand of file src/arXiv_src_0107_001.tar successful
Dowloand of file src/arXiv_src_0108_001.tar successful
Dowloand o

In [45]:
import urllib.request as req
url = 'http://export.arxiv.org/api/query?search_query=id:1804.00020&start=0&max_results=5'
data = req.urlopen(url).read()
root = ET.fromstring(data)
res = root.getchildren()[-1]
res.find('title').text

AttributeError: 'NoneType' object has no attribute 'text'

In [38]:
for el in res.getchildren():
    print(el.tag)

{http://www.w3.org/2005/Atom}id
{http://www.w3.org/2005/Atom}updated
{http://www.w3.org/2005/Atom}published
{http://www.w3.org/2005/Atom}title
{http://www.w3.org/2005/Atom}summary
{http://www.w3.org/2005/Atom}author
{http://www.w3.org/2005/Atom}author
{http://arxiv.org/schemas/atom}comment
{http://www.w3.org/2005/Atom}link
{http://www.w3.org/2005/Atom}link
{http://arxiv.org/schemas/atom}primary_category
{http://www.w3.org/2005/Atom}category
